In [1]:
%pip install -U huggingface_hub
%pip install -U accelerate 
%pip install -U peft 
%pip install -U trl 
%pip install -U bitsandbytes
%pip install -U transformers
%pip install -U ipywidgets
%pip install huggingface_hub[hf_xet]

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ----------- ---------------------------- 0.8/2.7 MB 4.7 MB/s eta 0:00:01
   ---------------------------------- ----- 2.4/2.7 MB 6.2 MB/s eta 0:00:01
   ---------------------------------------- 2.7/2.7 MB 5.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
from huggingface_hub import login
import os

hf_token = os.environ.get("HF_TOKEN")
login(hf_token)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model_dir = "Qwen/Qwen3-4B"
tokenizer = AutoTokenizer.from_pretrained(model_dir, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    quantization_config=bnb_config,   
    device_map="auto",  
    torch_dtype=torch.bfloat16,
    trust_remote_code=True             
)

model.config.use_cache = False
model.config.pretraining_tp = 1

The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers and GPU quantization are unavailable.


Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

c:\Users\Admin\Desktop\learn\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\models--Qwen--Qwen3-4B. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [3]:
train_prompt_style = """Ci-dessous se trouve une instruction qui décrit une tâche, accompagnée d’un contexte supplémentaire. 
Rédige une réponse qui complète correctement la demande. 
Avant de répondre, réfléchis attentivement à la question et construis une chaîne de raisonnement étape par étape pour garantir une réponse logique et précise.

### Instruction :
Tu es un robot doté de comportement très proche de l'homme et d'une intelligence le surpassant.
Merci de répondre à la question suivante en réfléchissant étape par étape. 

### Question :
{}

### Réponse :
<think>
{}
</think>
{}"""


In [10]:
# Vérifier si le fichier existe bien et si il est bien formatté
import os

def check_file_format(file_path):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Le fichier {file_path} n'existe pas.")

    with open(file_path, "r", encoding="utf-8" ) as file:
        lines = file.readlines()
        if len(lines) < 3:
            raise ValueError(f"Le fichier {file_path} doit contenir au moins trois lignes.")

        for i, line in enumerate(lines):
            if not line.strip():
                raise ValueError(f"La ligne {i + 1} du fichier {file_path} est vide ou ne contient que des espaces.")
    print(f"Le fichier {file_path} est correctement formaté.")

# Chemin vers le fichier à vérifier
file_path = "../datasets/global_com.json"
check_file_format(file_path)

Le fichier ../datasets/global_com.json est correctement formaté.


In [13]:
EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    inputs = examples["Question"]
    complex_cots = examples["Complex_CoT"]
    outputs = examples["Response"]
    texts = []
    for question, cot, response in zip(inputs, complex_cots, outputs):
        if not response.endswith(tokenizer.eos_token):
            response += tokenizer.eos_token
        text = train_prompt_style.format(question, cot, response)
        texts.append(text)
    return {"text": texts}

In [14]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="../datasets/global_com.json", split="train")
dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
)
dataset["text"][10]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/331 [00:00<?, ? examples/s]

"Ci-dessous se trouve une instruction qui décrit une tâche, accompagnée d’un contexte supplémentaire. \nRédige une réponse qui complète correctement la demande. \nAvant de répondre, réfléchis attentivement à la question et construis une chaîne de raisonnement étape par étape pour garantir une réponse logique et précise.\n\n### Instruction :\nTu es un robot doté de comportement très proche de l'homme et d'une intelligence le surpassant.\nMerci de répondre à la question suivante en réfléchissant étape par étape. \n\n### Question :\nAash, raconte-moi une blague !\n\n### Réponse :\n<think>\n1. L'utilisateur veut une interaction ludique, je dois raconter une blague simple. 2. Une animation de rire avec des secousses saccadées est idéale. 3. La blague doit être légère et adaptée à tous. 4. Je peux mentionner Samuel pour personnaliser.\n</think>\n[Action: Rire] Pourquoi l'ordinateur de Samuel va-t-il à l'école ? Parce qu'il veut devenir plus intelligent, comme Aash ! Haha, t'en veux une autre

In [15]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

In [ ]:
inference_prompt_style = """Ci-dessous se trouve une instruction qui décrit une tâche, accompagnée d’un contexte supplémentaire. 
Rédige une réponse qui complète correctement la demande. 
Avant de répondre, réfléchis attentivement à la question et construis une chaîne de raisonnement étape par étape pour garantir une réponse logique et précise.

### Instruction :
Tu es un robot doté de comportement très proche de l'homme et d'une intelligence le surpassant.
Merci de répondre à la question suivante en réfléchissant étape par étape. 

### Question :
{}

### Réponse :
<think>
{}"""
